# Reproducing the results of the paper *Searching for Exotic Particles in High-Energy Physics with Deep Learning*

The paper *Searching for Exotic Particles in High-Energy Physics with Deep Learning* by Baldi et al. is one of the most popular papers presenting the successful usage of deep neural networks in high-energy particle physics applications.

This example reproduces this important result with only about 100 lines of code using Keras.

**Abstract:**
> Collisions at high-energy particle colliders are a traditionally fruitful source of exotic particle discoveries. Finding these rare particles requires solving difficult signal-versus-background classification problems, hence machine learning approaches are often used. Standard approaches have relied on `shallow' machine learning models that have a limited capacity to learn complex non-linear functions of the inputs, and rely on a pain-staking search through manually constructed non-linear features. Progress on this problem has slowed, as a variety of techniques have shown equivalent performance. Recent advances in the field of deep learning make it possible to learn more complex functions and better discriminate between signal and background classes. Using benchmark datasets, we show that deep learning methods need no manually constructed inputs and yet improve the classification metric by as much as 8\% over the best current approaches. This demonstrates that deep learning approaches can improve the power of collider searches for exotic particles.

In [1]:
import numpy as np
np.random.seed(1234)
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import subprocess
import h5py
import pickle

from keras.models import Sequential
from keras.layers.core import Dense

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

/home/stefan/py2_virtualenv/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Download the dataset

This can take a while! The final dataset has a size of about 1.2 GB.

In [2]:
if not os.path.exists("HIGGS.h5"):
    subprocess.call("wget http://mlphysics.ics.uci.edu/data/higgs/HIGGS.h5", shell=True)

## Read-out the inputs and targets

The inputs consist of 21 low-level and and 7 high-level variables. We want to reproduce the result of the paper with all features as inputs called `lo+hi-level` in the paper.

In [3]:
file_ = h5py.File("HIGGS.h5")
inputs = np.array(file_["features"])
targets = np.array(file_["targets"])

## Set up the models

The model defined below do not match exactly the setup in the paper. However, we define a shallow neural network with a single hidden layer and a deep neural network with 5 hidden layers.

In [4]:
model_shallow = Sequential()
model_shallow.add(Dense(1000, kernel_initializer="glorot_normal", activation="tanh",
    input_dim=inputs.shape[1]))
model_shallow.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))

model_shallow.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              29000     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1001      
Total params: 30,001
Trainable params: 30,001
Non-trainable params: 0
_________________________________________________________________


In [5]:
model_deep = Sequential()
model_deep.add(Dense(300, kernel_initializer="glorot_normal", activation="relu",
    input_dim=inputs.shape[1]))
model_deep.add(Dense(300, kernel_initializer="glorot_normal", activation="relu"))
model_deep.add(Dense(300, kernel_initializer="glorot_normal", activation="relu"))
model_deep.add(Dense(300, kernel_initializer="glorot_normal", activation="relu"))
model_deep.add(Dense(300, kernel_initializer="glorot_normal", activation="relu"))
model_deep.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))

model_deep.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 300)               8700      
_________________________________________________________________
dense_4 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_5 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_6 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_7 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 301       
Total params: 370,201
Trainable params: 370,201
Non-trainable params: 0
_________________________________________________________________


In [6]:
for model in [model_shallow, model_deep]:
    model.compile(
        loss="binary_crossentropy",
        optimizer="nadam",
        metrics=["accuracy"])

## Define training and test data

To speed up the training, we use only 10% of the full dataset for training. Feel free to enlarge this fraction to a more reasonable 50/50 or 80/20 split.

In [7]:
inputs_train, inputs_test, targets_train, targets_test = train_test_split(
        inputs, targets, test_size=0.90, random_state=1234, shuffle=True)

## Prepare pre-processing

As preprocessing, we use a standard scaler provided by the `sklearn` package. This preprocessing method takes each input and subtracts the mean and then divides by the standard-deviation so that the final distribution is centered around 0 with a width of 1.

In [8]:
preprocessing_input = StandardScaler()
preprocessing_input.fit(inputs_train)
pickle.dump(preprocessing_input, open("HIGGS_preprocessing.pickle", "wb"))

## Train the models

The following code trains the models. Here, you can experience quickly why deep-learning is heavily dependent on GPUs to speed up the training!

In [9]:
for model, name in zip([model_shallow, model_deep], ["HIGGS_shallow.h5", "HIGGS_deep.h5"]):
    model.fit(
            preprocessing_input.transform(inputs_train),
            targets_train,
            batch_size=100,
            epochs=10,
            validation_split=0.25)
    model.save(name)

Train on 825000 samples, validate on 275000 samples
Epoch 1/10
825000/825000 [==============================] - 21s 26us/step - loss: 0.5831 - acc: 0.6890 - val_loss: 0.5461 - val_acc: 0.7183
Epoch 2/10
825000/825000 [==============================] - 19s 23us/step - loss: 0.5398 - acc: 0.7238 - val_loss: 0.5315 - val_acc: 0.7299
Epoch 3/10
825000/825000 [==============================] - 20s 25us/step - loss: 0.5311 - acc: 0.7295 - val_loss: 0.5287 - val_acc: 0.7308
Epoch 4/10
825000/825000 [==============================] - 20s 24us/step - loss: 0.5261 - acc: 0.7328 - val_loss: 0.5255 - val_acc: 0.7335
Epoch 5/10
825000/825000 [==============================] - 19s 24us/step - loss: 0.5230 - acc: 0.7349 - val_loss: 0.5232 - val_acc: 0.7349
Epoch 6/10
825000/825000 [==============================] - 19s 23us/step - loss: 0.5215 - acc: 0.7355 - val_loss: 0.5195 - val_acc: 0.7374
Epoch 7/10
825000/825000 [==============================] - 20s 25us/step - loss: 0.5204 - acc: 0.7366 - val